# Import Libraries

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import warnings
warnings.filterwarnings("ignore")


# Create Engine

In [2]:
# Define your connection string
connection_string = (
    "postgresql+psycopg2://neondb_owner:a9Am7Yy5r9_T7h4OF2GN" ### note that password changes all of the time, this is just an example, double check password with team lead
    "@ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech/neondb"
    "?sslmode=require&channel_binding=require"
)

In [3]:
# 2. Create SQLAlchemy engine
engine = create_engine(connection_string)

# csv laden

Relevant columns:
- DBN: relational key for database integration
- Num of SAT Test Takers: provides context for the average scores
- SAT Critical Reading Avg. Score: core performance metric
- SAT Math Avg. Score: core performance metric
- SAT Writing Avg. Score: core performance metric

Irrelevant columns:
- SCHOOL NAME: already available in the high school directory
- SAT Critical Readng Avg. Score: duplicate column with inconsistent naming
- internal_school_id: DBN serves as the primary identifier
- contact_extension: no analytical value for this task
- pct_students_tested: not required for the core SAT score analysis
- academic_tier_rating: could be integrated in a separate project, but is out of scope here

In [4]:
df_sat_results = pd.read_csv("sat-results.csv")

In [5]:
df_sat_results.head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,SAT Critical Readng Avg. Score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363,355,218160,x345,78%,2.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366,383,268547,x234,NaN,3.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370,377,236446,x123,NaN,3.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359,414,427826,x123,92%,4.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384,390,672714,x123,92%,2.0


In [6]:
df_sat_results.shape

(493, 11)

In [7]:
df_sat_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   DBN                              493 non-null    object 
 1   SCHOOL NAME                      493 non-null    object 
 2   Num of SAT Test Takers           493 non-null    object 
 3   SAT Critical Reading Avg. Score  493 non-null    object 
 4   SAT Math Avg. Score              493 non-null    object 
 5   SAT Writing Avg. Score           493 non-null    object 
 6   SAT Critical Readng Avg. Score   493 non-null    object 
 7   internal_school_id               493 non-null    int64  
 8   contact_extension                388 non-null    object 
 9   pct_students_tested              376 non-null    object 
 10  academic_tier_rating             402 non-null    float64
dtypes: float64(1), int64(1), object(9)
memory usage: 42.5+ KB


In [8]:
df_sat_results.describe()

,internal_school_id,academic_tier_rating
count,493.000000,402.000000
mean,562172.943205,2.564677
std,262138.627055,1.126443
min,101855.000000,1.000000
25%,332013.000000,2.000000
50%,587220.000000,3.000000
75%,782993.000000,4.000000
max,999398.000000,4.000000


In [9]:
df_sat_results_cleaned = df_sat_results[
    [
        "DBN", "Num of SAT Test Takers", "SAT Critical Reading Avg. Score", "SAT Math Avg. Score","SAT Writing Avg. Score"
    ]
]

df_sat_results_cleaned.head()

,DBN,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
0,01M292,29,355,404,363
1,01M448,91,383,423,366
2,01M450,70,377,402,370
3,01M458,7,414,401,359
4,01M509,44,390,433,384


## Data Cleaning

### Column names

In [10]:
# Clean Column names
df_sat_results_cleaned.columns = (
    df_sat_results_cleaned.columns
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(".", "", regex=False)
)


### DBN

In [11]:
# Clean dbn column
df_sat_results_cleaned["dbn"] = (
    df_sat_results_cleaned["dbn"]
    .astype(str)
    .str.strip()
)

### Numeric columns

In [12]:
df_sat_results_cleaned.head()

,dbn,num_of_sat_test_takers,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score
0,01M292,29,355,404,363
1,01M448,91,383,423,366
2,01M450,70,377,402,370
3,01M458,7,414,401,359
4,01M509,44,390,433,384


#### Text for none numeric values

- Every column contains "s", in general that means something like suppressed, small sample size or not reported
- replaced with "NaN"

In [13]:
print(df_sat_results_cleaned["num_of_sat_test_takers"].isnull().sum())
print(df_sat_results_cleaned["sat_critical_reading_avg_score"].isnull().sum())
print(df_sat_results_cleaned["sat_math_avg_score"].isnull().sum())
print(df_sat_results_cleaned["sat_writing_avg_score"].isnull().sum())

0
0
0
0


In [14]:
col = "num_of_sat_test_takers"

non_numeric = df_sat_results_cleaned[
    pd.to_numeric(df_sat_results_cleaned[col], errors="coerce").isna()
][col]

non_numeric.unique()


array(['s'], dtype=object)

In [15]:
col = "sat_critical_reading_avg_score"

non_numeric = df_sat_results_cleaned[
    pd.to_numeric(df_sat_results_cleaned[col], errors="coerce").isna()
][col]

non_numeric.unique()

array(['s'], dtype=object)

In [16]:
col = "sat_math_avg_score"

non_numeric = df_sat_results_cleaned[
    pd.to_numeric(df_sat_results_cleaned[col], errors="coerce").isna()
][col]

non_numeric.unique()

array(['s'], dtype=object)

In [17]:
col = "sat_writing_avg_score"

non_numeric = df_sat_results_cleaned[
    pd.to_numeric(df_sat_results_cleaned[col], errors="coerce").isna()
][col]

non_numeric.unique()

array(['s'], dtype=object)

In [18]:
numeric_cols = [
    "num_of_sat_test_takers",
    "sat_critical_reading_avg_score",
    "sat_math_avg_score",
    "sat_writing_avg_score",
]


In [19]:
# Convert all numeric SAT-related columns and replace suppressed values ('s') with NaN
for col in numeric_cols:
    df_sat_results_cleaned[col] = (
        df_sat_results_cleaned[col]
        .replace("s", None)    
    )

    df_sat_results_cleaned[col] = pd.to_numeric(
        df_sat_results_cleaned[col],
        errors="coerce"
    )


In [20]:
df_sat_results_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dbn                             493 non-null    object 
 1   num_of_sat_test_takers          435 non-null    float64
 2   sat_critical_reading_avg_score  435 non-null    float64
 3   sat_math_avg_score              435 non-null    float64
 4   sat_writing_avg_score           435 non-null    float64
dtypes: float64(4), object(1)
memory usage: 19.4+ KB


In [21]:
df_sat_results_cleaned[
    df_sat_results_cleaned["num_of_sat_test_takers"].isna()
].head()

,dbn,num_of_sat_test_takers,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score
22,02M392,NaN,NaN,NaN,NaN
23,02M393,NaN,NaN,NaN,NaN
25,02M399,NaN,NaN,NaN,NaN
38,02M427,NaN,NaN,NaN,NaN
40,02M437,NaN,NaN,NaN,NaN


## Scores validieren

- Average score achieved in the Critical Reading section (valid: 200–800)
- Average score achieved in the Math section (valid: 200–800)
- Average score achieved in the Writing section (valid: 200–800)

In [22]:
score_cols = [
    "sat_critical_reading_avg_score",
    "sat_math_avg_score",
    "sat_writing_avg_score",
]

In [23]:
for col in score_cols:
    print(f"\nInvalid values in {col}:")
    display(
        df_sat_results_cleaned.loc[
            (df_sat_results_cleaned[col] < 200) |
            (df_sat_results_cleaned[col] > 800),
            ["dbn", col]
        ]
    )


Invalid values in sat_critical_reading_avg_score:


,dbn,sat_critical_reading_avg_score



Invalid values in sat_math_avg_score:


,dbn,sat_math_avg_score
80,03M415,850.0
181,10X225,-10.0
288,15K656,999.0
422,28Q470,999.0
434,29Q283,1100.0



Invalid values in sat_writing_avg_score:


,dbn,sat_writing_avg_score


In [24]:
# Replace invalid SAT scores (outside 200–800 range) with NaN
for col in score_cols:
    df_sat_results_cleaned.loc[
        (df_sat_results_cleaned[col] < 200) |
        (df_sat_results_cleaned[col] > 800),
        col
    ] = None


In [25]:
df_sat_results_cleaned[score_cols].describe()
df_sat_results_cleaned.isna().sum()

dbn                                0
num_of_sat_test_takers            58
sat_critical_reading_avg_score    58
sat_math_avg_score                63
sat_writing_avg_score             58
dtype: int64

In [26]:
df_sat_results_cleaned.head()

,dbn,num_of_sat_test_takers,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score
0,01M292,29.0,355.0,404.0,363.0
1,01M448,91.0,383.0,423.0,366.0
2,01M450,70.0,377.0,402.0,370.0
3,01M458,7.0,414.0,401.0,359.0
4,01M509,44.0,390.0,433.0,384.0


In [27]:
df_sat_results_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dbn                             493 non-null    object 
 1   num_of_sat_test_takers          435 non-null    float64
 2   sat_critical_reading_avg_score  435 non-null    float64
 3   sat_math_avg_score              430 non-null    float64
 4   sat_writing_avg_score           435 non-null    float64
dtypes: float64(4), object(1)
memory usage: 19.4+ KB


## Drop duplicates

In [28]:
df_sat_results_cleaned = df_sat_results_cleaned.drop_duplicates(subset=["dbn"])
df_sat_results_cleaned = df_sat_results_cleaned.dropna(subset=["dbn"])


In [29]:
df_sat_results_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 478 entries, 0 to 477
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dbn                             478 non-null    object 
 1   num_of_sat_test_takers          421 non-null    float64
 2   sat_critical_reading_avg_score  421 non-null    float64
 3   sat_math_avg_score              416 non-null    float64
 4   sat_writing_avg_score           421 non-null    float64
dtypes: float64(4), object(1)
memory usage: 22.4+ KB


## Save cleaned csv

In [30]:
df_sat_results_cleaned.to_csv('cleaned_sat_results.csv', index=False)

In [31]:
df_sat_results_cleaned.to_sql(
    name='laura_sat_results',       
    con=engine,     
    schema='nyc_schools',
    if_exists='append',    
    index=False            
)

478

# Database Connection

In [32]:
# DB connection setup using hardcoded credentials (for onboarding only)
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="a9Am7Yy5r9_T7h4OF2GN",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require",
)
cur = conn.cursor()

# Databases test Querys - Overview

In [33]:
query = "SELECT * FROM nyc_schools.high_school_directory;"
df_high_school_directory = pd.read_sql(query, conn)
df_high_school_directory.head()

,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,NaN,NaN,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,NaN,NaN,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,NaN,NaN,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,NaN,NaN,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,NaN,NaN,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


In [34]:
query = "SELECT * FROM nyc_schools.school_safety_report;"
df_school_safety_report = pd.read_sql(query, conn)
df_school_safety_report.head()

,school_year,building_code,dbn,location_name,location_code,address,borough,geographical_district_code,register,building_name,...,postcode,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,_schools
0,2015-16,K006,17K006,P.S. 006,K006,43 SNYDER AVENUE,K,17.0,712,,...,11226.0,40.648590,-73.957138,14.0,40.0,794.0,3117230.0,3.051030e+09,Erasmus ...,1
1,2015-16,K005,16K005,P.S. 005 Dr. Ronald Mcnair,K005,820 HANCOCK STREET,K,16.0,231,,...,11233.0,40.685695,-73.922463,3.0,41.0,377.0,3039906.0,3.014900e+09,Stuyvesant Heights ...,1
2,2015-16,K003,13K003,P.S. 003 The Bedford Village,K003,50 JEFFERSON AVENUE,K,13.0,450,,...,11216.0,40.682663,-73.955338,3.0,36.0,227.0,3057406.0,3.019980e+09,Clinton Hill ...,1
3,2015-16,K002,75K141,P.S. K141,K141,655 PARKSIDE AVENUE,K,17.0,374,655 PARKSIDE AVENUE CONSOLIDATED LOCATION,...,11226.0,40.656183,-73.951583,9.0,40.0,802.0,3378201.0,3.050500e+09,Prospect Lefferts Gardens-Wingate ...,3
4,2015-16,K002,84K704,EXPLORE CHARTER SCHOOL(BS),K704,655 PARKSIDE AVENUE,K,17.0,529,655 PARKSIDE AVENUE CONSOLIDATED LOCATION,...,11226.0,40.656183,-73.951583,9.0,40.0,802.0,3378201.0,3.050500e+09,Prospect Lefferts Gardens-Wingate ...,3


In [35]:
query = "SELECT * FROM nyc_schools.school_demographics;"
df_school_demographics = pd.read_sql(query, conn)
df_school_demographics.head()

,dbn,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
0,01M015,P.S. 015 ROBERTO CLEMENTE,20052006,89.4,NaN,281,15,36,40,33,...,74,26.3,189,67.3,5,1.8,158,56.2,123,43.8
1,01M015,P.S. 015 ROBERTO CLEMENTE,20062007,89.4,NaN,243,15,29,39,38,...,68,28.0,153,63.0,4,1.6,140,57.6,103,42.4
2,01M015,P.S. 015 ROBERTO CLEMENTE,20072008,89.4,NaN,261,18,43,39,36,...,77,29.5,157,60.2,7,2.7,143,54.8,118,45.2
3,01M015,P.S. 015 ROBERTO CLEMENTE,20082009,89.4,NaN,252,17,37,44,32,...,75,29.8,149,59.1,7,2.8,149,59.1,103,40.9
4,01M015,P.S. 015 ROBERTO CLEMENTE,20092010,,96.5,208,16,40,28,32,...,67,32.2,118,56.7,6,2.9,124,59.6,84,40.4


In [36]:
query = "SELECT * FROM nyc_schools.laura_sat_results;"
df_sat_results_new = pd.read_sql(query, conn)
df_sat_results_new.head()

,dbn,num_of_sat_test_takers,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score
0,01M292,29.0,355.0,404.0,363.0
1,01M448,91.0,383.0,423.0,366.0
2,01M450,70.0,377.0,402.0,370.0
3,01M458,7.0,414.0,401.0,359.0
4,01M509,44.0,390.0,433.0,384.0


## Testjoin

In [37]:
query = """
SELECT
    sr.dbn,
    hsd.school_name,
    hsd.borough,
    sr.num_of_sat_test_takers,
    sr.sat_critical_reading_avg_score,
    sr.sat_math_avg_score,
    sr.sat_writing_avg_score
FROM nyc_schools.laura_sat_results sr
LEFT JOIN nyc_schools.high_school_directory hsd
    ON sr.dbn = hsd.dbn;
"""

df_joined = pd.read_sql(query, conn)
df_joined.head()


,dbn,school_name,borough,num_of_sat_test_takers,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score
0,01M292,Henry Street School for International Studies,Manhattan,29.0,355.0,404.0,363.0
1,01M448,University Neighborhood High School,Manhattan,91.0,383.0,423.0,366.0
2,01M450,East Side Community School,Manhattan,70.0,377.0,402.0,370.0
3,01M458,None,None,7.0,414.0,401.0,359.0
4,01M509,Marta Valle High School,Manhattan,44.0,390.0,433.0,384.0
